Implement batch gradient descent with early stopping for softmax regression without using Scikit-Learn, only NumPy. Use it on a classification task such as the iris dataset.

In [ ]:
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
X = iris.data
y = iris.target

In [196]:
X_with_bias = np.c_[np.ones(len(X)), X]

y_one_hot = np.zeros((len(y), 3))
y_one_hot[np.arange(len(y)), y] = 1

In [197]:
test_ratio = 0.2
validation_ratio = 0.2

np.random.seed(42)

total_size = len(X)
test_size = int(test_ratio * total_size)
validation_size = int(validation_ratio * total_size)
train_size = total_size - test_size - validation_size

random_indices = np.random.permutation(total_size)

X_train = X_with_bias[random_indices[:train_size]]
y_train = y_one_hot[random_indices[:train_size]]

X_valid = X_with_bias[random_indices[train_size:train_size+validation_size]]
y_valid = y_one_hot[random_indices[train_size:train_size+validation_size]]

X_test = X_with_bias[random_indices[train_size+validation_size:]]
y_test = y_one_hot[random_indices[train_size+validation_size:]]

In [198]:
mean = X_train[:, 1:].mean(axis=0)
std = X_train[:, 1:].std(axis=0)
X_train[:, 1:] = (X_train[:, 1:] - mean) / std
X_valid[:, 1:] = (X_valid[:, 1:] - mean) / std
X_test[:, 1:] = (X_test[:, 1:] - mean) / std

In [199]:
eta = 0.1 
n_epochs = 1000
alpha = 0.01

m = len(X_train)
inputs = len(X_train[0])
outputs = len(np.unique(y))
 
theta = np.random.randn(inputs, outputs)

def probability(softmax_scores):
    exps = np.exp(softmax_scores)
    exp_sums = exps.sum(axis=1, keepdims=True)
    return exps / exp_sums

def probability_k(softmax_scores, k):
    return (np.exp(softmax_scores[k])) / sum([np.exp(s) for s in softmax_scores])

def accuracy(theta):
    sm_scores = X_valid @ theta
    y_proba = probability(sm_scores)
    y_predict = y_proba.argmax(axis=1)
    y_valid_1d = y_valid.argmax(axis=1)

    return (y_predict == y_valid_1d).mean()

checkpoint = int(n_epochs / 10)

for epoch in range(n_epochs):
    if epoch % checkpoint == 0:
        print(f"Training data: {epoch / n_epochs * 100 :.0f}%",
              f"Accuracy: {accuracy(theta) * 100 :.3f}%")
        
    softmax_scores = X_train @ theta
    y_proba = probability(softmax_scores)
    reg_gradient = alpha * theta
    gradients = (1 / m) * X_train.T @ (y_proba - y_train) + reg_gradient
    theta = theta - (eta * gradients)

Training data: 0% Accuracy: 0.000%
Training data: 10% Accuracy: 86.667%
Training data: 20% Accuracy: 86.667%
Training data: 30% Accuracy: 90.000%
Training data: 40% Accuracy: 93.333%
Training data: 50% Accuracy: 93.333%
Training data: 60% Accuracy: 93.333%
Training data: 70% Accuracy: 93.333%
Training data: 80% Accuracy: 93.333%
Training data: 90% Accuracy: 93.333%


In [200]:
theta

array([[-0.39072784,  0.96005921, -1.605765  ],
       [-0.7154986 ,  0.26480741,  0.48918194],
       [ 1.20162228, -0.25405516, -0.24278623],
       [-1.30635205,  0.56604163,  1.42493294],
       [-1.41502116, -0.57502337,  2.02946397]])

In [201]:
sm_scores = X_valid @ theta
y_proba = probability(sm_scores)
y_predict = y_proba.argmax(axis=1)
y_valid_2 = y_valid.argmax(axis=1)

acc = (y_predict == y_valid_2).mean()

In [202]:
acc

np.float64(0.9333333333333333)